# Derive models of drone

Do all imports.

In [31]:
import sympy as sym
import numpy as np

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

## Dynamic model

Define physical parameters.

Derive the equations of motion:

In [117]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_phi, w_theta, w_psi = sym.symbols('w_phi, w_theta, w_psi')
w_in_body = sym.Matrix([w_phi, w_theta, w_psi])

# rotor components of angular velcoity 
w_m1, w_m2, w_m3, w_m4 = sym.symbols('w_m1, w_m2, w_m3, w_m4')
R_angular = sym.Matrix([w_m1, w_m2, w_m3, w_m4])

# components of net rotor torque
tau_phi, tau_theta, tau_psi = sym.symbols('tau_phi, tau_theta, tau_psi')
tau_in_body = sym.Matrix([tau_phi, tau_theta, tau_psi])

# component of individual rotor torque
t_1, t_2, t_3, t_4 = sym.symbols('t_1, t_2, t_3, t_4')
R_torques = sym.Matrix([t_1, t_2, t_3, t_4])

# component of inertia 
Ixx, Iyy, Izz = sym.symbols('Ixx, Iyy, Izz')
I = sym.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])
# component of rotor inertia 
Im = sym.symbols('I_m')

# net rotor force
f_z = sym.symbols('f_z')

k = sym.symbols('k')
b = sym.symbols('b')
m = sym.symbols('m')
g = sym.symbols('g')
l = sym.symbols('l')

Fg = sym.Matrix([0,0,g*m])




In [118]:
# rotation matrices
Rx = sym.Matrix([[1,           0,              0],
                 [0, sym.cos(phi), -sym.sin(phi)], 
                 [0, sym.sin(phi), sym.cos(phi)]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], 
                 [0,              1,              0], 
                 [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], 
                 [sym.sin(psi), sym.cos(psi), 0], 
                 [0,            0,            1]])
RBtoW = Rz @ Ry @ Rx
RWtoB = RBtoW.T

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
RAtoB = sym.Matrix.hstack(ex, Rx.T @ ey, (Ry@Rx).T@ez)
RAtoB_inv = RAtoB.inv()

RBtoW

Matrix([
[cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi),  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)],
[sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)],
[        -sin(theta),                              sin(phi)*cos(theta),                               cos(phi)*cos(theta)]])

In [119]:
def symSum(vector):
    total_sum = 0
    for i in vector:
        total_sum += i
    return total_sum

In [120]:
def RotorAcceleration (R_angular, R_torques):
    F_drag = R_angular.applyfunc(lambda x: x**2 * b) 
    F_motor = R_torques 
    dWm = (F_motor - F_drag) / Im 
    return dWm 

def RotorForces(R_angular, R_torques): 
    w_m1 = R_angular[0] 
    w_m2 = R_angular[1] 
    w_m3 = R_angular[2] 
    w_m4 = R_angular[3] 

    individual_T = R_angular.applyfunc(lambda x: x**2 * k) 
    thrust = sym.simplify(symSum(individual_T))  # Ensure to evaluate the summation 
    thrust_T = sym.Matrix([0, 0, thrust]) 


    tau_phi = l*k*(-w_m2**2 + w_m4**2) 
    tau_theta = l*k*(-w_m1**2 + w_m3**2) 
    dwm = RotorAcceleration(R_angular, R_torques) 
    [dwm1, dwm2, dwm3, dwm4] = dwm 
    Tm1 = -b*w_m1**2 - Im*dwm1 
    Tm2 = b*w_m2**2 + Im*dwm2 
    Tm3 = -b*w_m3**2 + -Im*dwm3 
    Tm4 = b*w_m4**2 + Im*dwm4 
    tau_psi = sym.simplify(Tm1+Tm2+Tm3+Tm4) 
    tau = sym.Matrix ([tau_phi, tau_theta, tau_psi]) 
    return thrust_T, tau, dwm 
    
print(RotorForces(R_angular, R_torques)) 

(Matrix([
[                                        0],
[                                        0],
[k*(w_m1**2 + w_m2**2 + w_m3**2 + w_m4**2)]]), Matrix([
[k*l*(-w_m2**2 + w_m4**2)],
[k*l*(-w_m1**2 + w_m3**2)],
[  -t_1 + t_2 - t_3 + t_4]]), Matrix([
[(-b*w_m1**2 + t_1)/I_m],
[(-b*w_m2**2 + t_2)/I_m],
[(-b*w_m3**2 + t_3)/I_m],
[(-b*w_m4**2 + t_4)/I_m]]))


In [122]:
RWtoB = RBtoW.T
RAtoG = sym.simplify(RAtoB_inv)

thrust, tau, dwm = RotorForces(R_angular, R_torques)
F_thrust = sym.simplify(RBtoW @ thrust)
A_linear = sym.simplify((Fg + F_thrust) / m)
A_angular = sym.simplify(RAtoG @ (I.inv() @ tau))

result = sym.Matrix([v_in_body, A_linear, w_in_body, A_angular, dwm])
result


Matrix([
[                                                                                                                                                         v_x],
[                                                                                                                                                         v_y],
[                                                                                                                                                         v_z],
[                                                              k*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))*(w_m1**2 + w_m2**2 + w_m3**2 + w_m4**2)/m],
[                                                             -k*(sin(phi)*cos(psi) - sin(psi)*sin(theta)*cos(phi))*(w_m1**2 + w_m2**2 + w_m3**2 + w_m4**2)/m],
[                                                                                     (g*m + k*(w_m1**2 + w_m2**2 + w_m3**2 + w_m4**2)*cos(phi)*cos(theta))/m],
[                              